##Loading Utils

In [ ]:
!pip install sacrebleu
!pip install torchtext==0.6.0

In [2]:
%%capture
!pip install --upgrade sacrebleu sentencepiece

import json
import math
import random

import matplotlib.pyplot as plt
import numpy as np
import sacrebleu
import sentencepiece
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
import tqdm.notebook

In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

Using device: cuda


##Loading and Processing Data

In [4]:
extensions = [".de", ".en"]
source_field = torchtext.data.Field(tokenize = lambda x: x)
target_field = torchtext.data.Field(tokenize = lambda x: x)
training_data, validation_data, test_data = torchtext.datasets.Multi30k.splits(
    extensions, [source_field, target_field], root = "/content/drive/MyDrive/NLP/NMT"
)

In [5]:
print("Number of training examples:", len(training_data))
print("Number of validation examples:", len(validation_data))
print("Number of test examples:", len(test_data))
print()

for example in training_data[:10]:
  print(example.src)
  print(example.trg)
  print()

Number of training examples: 29000
Number of validation examples: 1014
Number of test examples: 1000

Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
Two young, White males are outside near many bushes.

Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.
Several men in hard hats are operating a giant pulley system.

Ein kleines Mädchen klettert in ein Spielhaus aus Holz.
A little girl climbing into a wooden playhouse.

Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.
A man in a blue shirt is standing on a ladder cleaning a window.

Zwei Männer stehen am Herd und bereiten Essen zu.
Two men are at the stove preparing food.

Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.
A man in green holds a guitar while the other man observes his shirt.

Ein Mann lächelt einen ausgestopften Löwen an.
A man is smiling at a stuffed lion

Ein schickes Mädchen spricht mit dem Handy während sie langsam die Straße entla

In [6]:
from torchtext.data.functional import sentencepiece_numericalizer
args = {
    "pad_id": 0,
    "bos_id": 1,
    "eos_id": 2,
    "unk_id": 3,
    "input": "/content/drive/MyDrive/NLP/NMT/multi30k/train.de,/content/drive/MyDrive/NLP/NMT/multi30k/train.en",
    "vocab_size": 8000,
    "model_prefix": "multi30k",
}

combined_args = " ".join(
    "--{}={}".format(key, value) for key, value in args.items())

sentencepiece.SentencePieceTrainer.Train(combined_args)

In [7]:
!head -n 30 multi30k.vocab

<pad>	0
<s>	0
</s>	0
<unk>	0
.	-2.72718
▁a	-3.21357
▁in	-3.43973
m	-3.78503
▁eine	-3.82141
▁A	-3.86856
s	-4.06457
▁Ein	-4.11399
,	-4.20405
▁the	-4.35217
▁und	-4.5704
▁mit	-4.57911
▁auf	-4.58144
▁on	-4.65674
n	-4.67038
▁Mann	-4.70521
▁is	-4.73988
▁man	-4.75331
▁and	-4.76404
▁	-4.76512
ing	-4.8072
▁of	-4.83344
▁einer	-4.86421
▁with	-4.93426
▁Eine	-4.98902
▁ein	-5.126


In [8]:
vocab = sentencepiece.SentencePieceProcessor()
vocab.Load("multi30k.model")

True

In [9]:
print("Vocabulary size:", vocab.GetPieceSize())
print()

for example in training_data[:3]:
  sentence = example.trg
  pieces = vocab.EncodeAsPieces(sentence)
  indices = vocab.EncodeAsIds(sentence)
  print(sentence)
  print(pieces)
  print(vocab.DecodePieces(pieces))
  print(indices)
  print(vocab.DecodeIds(indices))
  print()

piece = vocab.EncodeAsPieces("the")[0]
index = vocab.PieceToId(piece)
print(piece)
print(index)
print(vocab.IdToPiece(index))



Vocabulary size: 8000

Two young, White males are outside near many bushes.
['▁Two', '▁young', ',', '▁White', '▁males', '▁are', '▁outside', '▁near', '▁many', '▁bushes', '.']
Two young, White males are outside near many bushes.
[42, 54, 12, 2889, 2225, 36, 127, 173, 815, 3513, 4]
Two young, White males are outside near many bushes.

Several men in hard hats are operating a giant pulley system.
['▁Se', 'veral', '▁men', '▁in', '▁hard', '▁hats', '▁are', '▁operating', '▁a', '▁g', 'iant', '▁pull', 'e', 'y', '▁s', 'y', 'ste', 'm', '.']
Several men in hard hats are operating a giant pulley system.
[298, 240, 73, 6, 712, 730, 36, 3106, 5, 631, 1679, 583, 32, 96, 552, 96, 1076, 7, 4]
Several men in hard hats are operating a giant pulley system.

A little girl climbing into a wooden playhouse.
['▁A', '▁little', '▁girl', '▁climbing', '▁in', 'to', '▁a', '▁wooden', '▁play', 'house', '.']
A little girl climbing into a wooden playhouse.
[9, 132, 66, 500, 6, 112, 5, 542, 245, 4599, 4]
A little girl cli

In [10]:
pad_id = vocab.PieceToId("<pad>")
bos_id = vocab.PieceToId("<s>")
eos_id = vocab.PieceToId("</s>")
print(pad_id,"\n",bos_id,"\n",eos_id)

0 
 1 
 2


In [11]:
sentence = training_data[0].trg
indices = vocab.EncodeAsIds(sentence)
indices_augmented = [bos_id] + indices + [eos_id, pad_id, pad_id, pad_id]
print(vocab.DecodeIds(indices))
print(vocab.DecodeIds(indices_augmented))
print(vocab.DecodeIds(indices) == vocab.DecodeIds(indices_augmented))

Two young, White males are outside near many bushes.
Two young, White males are outside near many bushes.
True


## Baseline sequence-to-sequence model

In [12]:
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import LongTensor

In [13]:
def make_batch(sentences):

  batch = []

  for sentence in sentences:
    indices = vocab.EncodeAsIds(sentence)
    indices.insert(0,1)
    indices.append(2)
    batch.append(torch.tensor(indices))

  batch = LongTensor(pad_sequence(batch)) # L x N
  batch = batch.to(device)
  return batch


def make_batch_iterator(dataset, batch_size, shuffle = False):

  examples = list(dataset)
  if shuffle:
    random.shuffle(examples)

  for start_index in range(0, len(examples), batch_size):
    example_batch = examples[start_index: start_index + batch_size]
    source_sentences = [example.src for example in example_batch]
    target_sentences = [example.trg for example in example_batch]
    yield make_batch(source_sentences), make_batch(target_sentences)

test_batch = make_batch(["a test input", "a longer input than the first"])
print("Example batch tensor:")
print(test_batch)
assert test_batch.shape[1] == 2
assert test_batch[0, 0] == bos_id
assert test_batch[0, 1] == bos_id
assert test_batch[-1, 0] == pad_id
assert test_batch[-1, 1] == eos_id

Example batch tensor:
tensor([[   1,    1],
        [   5,    5],
        [3966,  354],
        [   6,   60],
        [ 236,    6],
        [ 698,  236],
        [   2,  698],
        [   0, 5285],
        [   0,   13],
        [   0, 3759],
        [   0,    2]], device='cuda:0')


In [14]:
class Seq2seqBaseline(nn.Module):
  def __init__(self):
    super().__init__()
    self.embedding_layer_en = nn.Embedding(8000, 256)
    self.embedding_layer_de = nn.Embedding(8000, 256)
    self.encoder_lstm = nn.LSTM(input_size = 256, hidden_size = 256, num_layers = 2, dropout = 0.5, bidirectional = True)
    self.decoder_lstm = nn.LSTM(input_size = 256, hidden_size = 256, num_layers = 2, dropout = 0.5)
    self.dropout = nn.Dropout(0.5)
    self.linear = nn.Linear(256, 8000)
    self.criterion = nn.CrossEntropyLoss()

  def encode(self,source):

    lengths = torch.sum(source != pad_id, axis=0)
    embed = self.embedding_layer_en(source) # L x N x H
    packed_input = pack_padded_sequence(embed, lengths.cpu(),enforce_sorted=False)
    packed_output, hidden = self.encoder_lstm(packed_input) # L x N x 2 * H, 2 * L x N x H
    output, input_sizes = pad_packed_sequence(packed_output) # L x N x 2 * H
    h, c = hidden # 2*L x N x H
    h = h[:2, :, :] + h[2:, :, :] # L x N x H
    c = c[:2, :, :] + c[2:, :, :] # L x N x H
    hidden = (h,c)
    encoder_mask = (source != pad_id).permute(1,0)
    return (output, encoder_mask, hidden)

  def decode(self, decoder_input, intial_hidden, encoder_output, encoder_mask):

    #Baseline model - No attention
    del encoder_output
    del encoder_mask

    embeds = F.embedding(decoder_input, self.linear.weight)
    if(len(embeds.size()) == 2):
      embeds = embeds.view(1, embeds.size(0), embeds.size(1))
    hidden = intial_hidden
    output, hidden = self.decoder_lstm(embeds, hidden) # L x N x H, L x N x H
    output = self.dropout(output)
    output = F.log_softmax(self.linear(output), dim = 1)
    return (output, hidden, None)


  def compute_loss(self, source, target):
    batch_size = target.shape[1]
    max_seq_len = target.shape[0]
    outputs = torch.zeros(max_seq_len, batch_size, 8000).to(device)
    encoder_output, encoder_mask, hidden = self.encode(source)
    decoder_input = target[0,:]
    for i in range(1, max_seq_len):
      decoder_output, hidden, attention_weigths = self.decode(decoder_input, hidden, encoder_output, encoder_mask)
      outputs[i] = decoder_output
      decoder_input = target[i]

    logits = outputs[1:]
    outputs = logits.view(-1, 8000)
    target = target[1:].view(-1)
    loss = self.criterion(outputs, target)
    return loss, logits

In [15]:
def train(model, num_epochs, batch_size, model_file):

  optimizer = torch.optim.Adam(model.parameters())
  best_accuracy = 0.0
  for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
    with tqdm.notebook.tqdm(
        make_batch_iterator(training_data, batch_size, shuffle=True),
        desc="epoch {}".format(epoch + 1),
        unit="batch",
        total=math.ceil(len(training_data) / batch_size)) as batch_iterator:
      model.train()
      total_loss = 0.0
      for i, (source, target) in enumerate(batch_iterator, start=1):
        optimizer.zero_grad()
        loss, _ = model.compute_loss(source, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        batch_iterator.set_postfix(mean_loss=total_loss / i)
      validation_perplexity, validation_accuracy = evaluate_next_token(
          model, validation_data)
      batch_iterator.set_postfix(
          mean_loss=total_loss / i,
          validation_perplexity=validation_perplexity,
          validation_token_accuracy=validation_accuracy)
      if validation_accuracy > best_accuracy:
        print(
            "Obtained a new best validation accuracy of {:.2f}, saving model "
            "checkpoint to {}...".format(validation_accuracy, model_file))
        torch.save(model.state_dict(), model_file)
        best_accuracy = validation_accuracy
  print("Reloading best model checkpoint from {}...".format(model_file))
  model.load_state_dict(torch.load(model_file))

def evaluate_next_token(model, dataset, batch_size=64):

  model.eval()
  total_cross_entropy = 0.0
  total_predictions = 0
  correct_predictions = 0
  with torch.no_grad():
    for source, target in make_batch_iterator(dataset, batch_size):
      decoder_input, decoder_target = target[:-1], target[1:]
      loss, logits = model.compute_loss(source, target)
      total_cross_entropy += loss.item()
      total_predictions += (decoder_target != pad_id).sum().item()
      correct_predictions += (
          (decoder_target != pad_id) &
          (decoder_target == logits.argmax(2))).sum().item()
  perplexity = math.exp(total_cross_entropy / total_predictions)
  accuracy = 100 * correct_predictions / total_predictions
  return perplexity, accuracy

In [32]:
num_epochs = 15
batch_size = 16
baseline_model = Seq2seqBaseline().to(device)
train(baseline_model, num_epochs, batch_size, "/content/drive/MyDrive/NLP/NMT/baseline_model.pt")

training:   0%|          | 0/15 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 31.00, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/baseline_model.pt...


epoch 2:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 36.73, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/baseline_model.pt...


...

Obtained a new best validation accuracy of 50.02, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/baseline_model.pt...


epoch 14:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 50.31, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/baseline_model.pt...


epoch 15:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 50.50, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/baseline_model.pt...
Reloading best model checkpoint from /content/drive/MyDrive/NLP/NMT/baseline_model.pt...


In [33]:
def predict_greedy(model, sentences, max_length=100):
  source = make_batch(sentences)
  batch_size = source.shape[1]
  max_seq_len = source.shape[0]
  outputs = torch.zeros(max_seq_len, batch_size, 1).to(device)
  encoder_output, encoder_mask, hidden = model.encode(source)
  decoder_input = source[0,:]

  for i in range(1, max_seq_len):
    decoder_output, hidden, attention_weigths = model.decode(decoder_input, hidden, encoder_output, encoder_mask)
    decoder_output = decoder_output.view(batch_size, 8000)
    decoder_output = torch.argmax(decoder_output, dim = 1)
    decoder_input = decoder_output.view(1, batch_size)
    decoder_output = decoder_output.view(batch_size, -1)
    outputs[i] = decoder_output

  outputs = outputs.view(max_seq_len,-1).int()
  translated_sentences = []
  for i in range(batch_size):
    indices = outputs[:,i]
    indices = indices.tolist()
    sentence = vocab.DecodeIds(indices)
    translated_sentences.append(sentence)

  return translated_sentences


def evaluate(model, dataset, batch_size=64, method="greedy"):
  assert method in {"greedy", "beam"}
  source_sentences = [example.src for example in dataset]
  target_sentences = [example.trg for example in dataset]
  model.eval()
  predictions = []
  with torch.no_grad():
    for start_index in range(0, len(source_sentences), batch_size):
      if method == "greedy":
        prediction_batch = predict_greedy(
            model, source_sentences[start_index:start_index + batch_size])
      else:
        prediction_batch = predict_beam(
            model, source_sentences[start_index:start_index + batch_size])
        prediction_batch = [candidates[0] for candidates in prediction_batch]
      predictions.extend(prediction_batch)
  return sacrebleu.corpus_bleu(predictions, [target_sentences]).score

In [34]:
print("Baseline model validation BLEU using greedy search:",
      evaluate(baseline_model, validation_data))

Baseline model validation BLEU using greedy search: 18.76028291824834


In [35]:
def show_predictions(model, num_examples=10, include_beam=False):
  source = []
  target = []
  for example in validation_data[:num_examples]:
    source.append(example.src)
    target.append(example.trg)
  prediction = predict_greedy(model, source)
  for i in range(num_examples):
    print("Input:")
    print(" ", source[i])
    print("Target:")
    print(" ", target[i])
    print("Greedy prediction:")
    print(" ", prediction[i])
    print()

In [36]:
print("Baseline model sample predictions:")
print()
show_predictions(baseline_model)

Baseline model sample predictions:

Input:
  Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
Target:
  A group of men are loading cotton onto a truck
Greedy prediction:
  A group of men are performing aschuhen.

Input:
  Ein Mann schläft in einem grünen Raum auf einem Sofa.
Target:
  A man sleeping in a green room on a couch.
Greedy prediction:
  A man sleeps in a park in a park.

Input:
  Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.
Target:
  A boy wearing headphones sits on a woman's shoulders.
Greedy prediction:
  Boy sitting alone on the sidewalk with a young girl.

Input:
  Zwei Männer bauen eine blaue Eisfischerhütte auf einem zugefrorenen See auf
Target:
  Two men setting up a blue ice fishing hut on an iced over lake
Greedy prediction:
  eine men eineschuhens on a couch, possibly soldiers are gathered in aschuhen eine.

Input:
  Ein Mann mit beginnender Glatze, der eine rote Rettungsweste trägt, sitzt in einem kleinen Boot.
Target:
  A balding man wea

## Sequence-to-Sequence model with attention

In [37]:
class Seq2seqAttention(Seq2seqBaseline):
  def __init__(self):
    super().__init__()
    self.linear_1 = nn.Linear(512, 256)
    self.linear_2 = nn.Linear(256, 8000)

  def decode(self, decoder_input, initial_hidden, encoder_output, encoder_mask):
    embeds = F.embedding(decoder_input, self.linear_2.weight)
    if(len(embeds.size()) == 2):
      embeds = embeds.view(1, embeds.size(0), embeds.size(1))

    hidden = initial_hidden
    output, hidden = self.decoder_lstm(embeds, hidden)
    encoder_output = encoder_output[:, :, :256] + encoder_output[:, :, 256:]
    attention_scores = torch.sum(output * encoder_output, dim = 2)
    attention_scores = attention_scores.t()
    attention_scores = attention_scores.masked_fill(encoder_mask == 0, -1e9)
    attention_weights = F.softmax(attention_scores, dim = 1).unsqueeze(1)
    context = attention_weights.bmm(encoder_output.transpose(0, 1))
    output = output.squeeze(0)
    context = context.squeeze(1)
    output = torch.cat((output, context), 1)
    output = self.linear_1(output)
    output = F.log_softmax(self.linear_2(output), dim = 1)
    return (output, hidden, attention_weights)

In [38]:
num_epochs = 10
batch_size = 16

attention_model = Seq2seqAttention().to(device)
train(attention_model, num_epochs, batch_size, "/content/drive/MyDrive/NLP/NMT/attention_model.pt")

training:   0%|          | 0/10 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 56.61, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/attention_model.pt...


epoch 2:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 63.08, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/attention_model.pt...


...

epoch 8:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 67.52, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/attention_model.pt...


epoch 9:   0%|          | 0/1813 [00:00<?, ?batch/s]

epoch 10:   0%|          | 0/1813 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 67.70, saving model checkpoint to /content/drive/MyDrive/NLP/NMT/attention_model.pt...
Reloading best model checkpoint from /content/drive/MyDrive/NLP/NMT/attention_model.pt...


In [39]:
print("Attention model validation BLEU using greedy search:",
      evaluate(attention_model, validation_data))
print()

Attention model validation BLEU using greedy search: 38.45073883830298



In [41]:
print("Attention model sample predictions:")
print()
show_predictions(attention_model)

Attention model sample predictions:

Input:
  Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
Target:
  A group of men are loading cotton onto a truck
Greedy prediction:
  A group of men unload logs on a truck

Input:
  Ein Mann schläft in einem grünen Raum auf einem Sofa.
Target:
  A man sleeping in a green room on a couch.
Greedy prediction:
  A man asleep on a couch in a green room.

Input:
  Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.
Target:
  A boy wearing headphones sits on a woman's shoulders.
Greedy prediction:
  A boy with headphones sitting on his shoulders of a woman.

Input:
  Zwei Männer bauen eine blaue Eisfischerhütte auf einem zugefrorenen See auf
Target:
  Two men setting up a blue ice fishing hut on an iced over lake
Greedy prediction:
  Two men are building a blue iced gated with a blue iced.

Input:
  Ein Mann mit beginnender Glatze, der eine rote Rettungsweste trägt, sitzt in einem kleinen Boot.
Target:
  A balding man wearing a red lif